# Incremental Evolution

There are some problems where the solution can be described as a set of small solutions that add up. In these cases, it may be interesting to carry out an incremental evolution. In it, a part of the individuals is blocked, while its other part evolves. After a few generations the blocked part is then released to evolve, evolving another piece of the solution.

Let's implement an evolutionary algorithm that performs incremental evolution.

In [1]:
import evolvepy as ep
import numpy as np
from matplotlib import pyplot as plt

# Fitness Function

Our problem will be to find the coefficients of a polynomial, in this case $y = 10 x^3 + 25 x^2$. This is not a problem that would need incremental evolution, but let's use it as an example.

The fitness function will be the error of our individual represented polynomial and the target:

$$error(individual) = \sqrt{\sum_{x=-100}^{100} (polynomial(x) - individual(x))^2}$$

Since we want to maximize the fitness, we will use $-error$.

In [2]:
import numba

@numba.jit
def f(x):
    return 10*np.power(x, 3) + 25*np.power(x, 2)

def fitness_function(individuals):
    individual = individuals[0]

    x = np.arange(-100, 100, 1)
    y = f(x)

    y_individual = individual["chr0"]*np.power(x, 3) + individual["chr1"]*np.power(x, 2)
     
    score = np.linalg.norm(y-y_individual)/1E7
    
    return -score

evaluator = ep.evaluator.FunctionEvaluator(fitness_function)

# Generator

Let's define a generator with elitism. Note that, unlike the previous examples, we are defining layers of type "FirstGenLayer". These layers generate the initial individuals and are normally created automatically by the Generator. We created two of these layers: the first will run in the first generation as expected, but it will just randomly generate the first chromosome, the second will be initialized with zeros. The second layer will be used to initialize the second chromosome in the middle of evolution (the parameter "run=False" indicates that it will not be executed right at the beginning).

We are also using "Block" layer. This layer will prevent our second chromosome from being altered by mutations or other operators.

In [27]:
descriptor = ep.generator.Descriptor([1, 1], [(-100.0, 100.0), (-100.0, 100.0)], types=[np.float32, np.float32])

# Blocks the second chromossome
block = ep.generator.Block("chr1")

mutation = ep.generator.mutation.NumericMutationLayer(ep.generator.mutation.sum_mutation, 1.0, 0.0, (-10.0, 10.0))
combine = ep.generator.CombineLayer(ep.generator.selection.tournament, ep.generator.crossover.one_point)
filter0 = ep.generator.FilterFirsts(95)
sort = ep.generator.Sort()
filter1 = ep.generator.FilterFirsts(5)
concat = ep.generator.Concatenate()

# The FirstGenLayers
first_gen0 = ep.generator.FirstGenLayer(descriptor, initialize_zeros=True, chromossome_names="chr0") #Generate only the first chromossome.
first_gen1 = ep.generator.FirstGenLayer(descriptor, chromossome_names="chr1", run=False) #Generate the second chromossome.

block.next = sort
sort.next = filter1
filter1.next = concat

block.next = mutation
mutation.next = combine
combine.next = filter0
filter0.next = concat

concat.next = first_gen0
first_gen0.next = first_gen1

gen = ep.generator.Generator(first_layer=block, last_layer=first_gen1)

# Evolver and Incremental Evolution

Here we are creating an Evolver with a dynamic mutation callback and the incremental evolution. The incremental evolution receives the generation in which the chromosome will be unlocked, the Block layer and the FirstGenLayer. It can also receive callbacks to block the execution before the unlock generation. We use this to prevent the dynamic mutation of changing mutation rates before we have all the chromosomes initially evolved.

In [18]:
dyn_mut = ep.callbacks.DynamicMutation([mutation.name], patience=3, refinement_steps=5, exploration_patience=0, exploration_steps=0, exploration_multiplier=1)
inc_evol = ep.callbacks.IncrementalEvolution(15, block_layer=block, first_gen_layer=first_gen1, callbacks=[dyn_mut])


evolver = ep.Evolver(gen, evaluator, 100, [inc_evol, dyn_mut])

# Results

Let's evolve for some generation and see the results

In [19]:
hist, last_pop = evolver.evolve(30)

In [ ]:
print("Best individual: ", last_pop[np.argmax(hist[-1])])

plt.plot(hist.max(axis=1))
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("Evolution history")
plt.legend(["Best"])
plt.show()

Looking at the evolution history, we can see how fitness took a leap in generation 15, in which we unlocked the second individual. After that, the generation is stuck for a few generations before activating the dynamic mutation.

![](Example5-Figure1.jpg)